# 2. Create an Incident from an Alert

Notebook to build all of the different stix objects associated with a Phishing Alert imitialising an Incident. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are six stages:

1. First setup the global parameters and retrieve the context memory
2. Create the objects from the phishing email Alert,
2. Add these objects to the `ObservedData` object
3. Then create the `Indicator` and get the user `Identity` object from context memory
4. Create the `Sighting` object with the `SightingAlert` extension
5. Create the Event and the Sequence objects
6. Create the Incident, with the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc), and add all of the objects to it


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).


![](img/Evidence_Alert.png)
![](img/Evidence_Alert_Flows.png)

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block,
)
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "nsmith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}
# Make the Variable to hold the Bundle List
bundle_list = []
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []
results_paths = {}

### A.4 Load the Context Memory

In [4]:
# 1. Set the Variables
context_identities = []
identities_file = "identity_bundle.json"
context_path = context_base + identities_file
#2. Load the file
if os.path.exists(context_path):
    with open(context_path, "r") as script_input:
        export_data = json.load(script_input)
        context_identities = export_data["objects"]

## B. Create the Objects to Document the Phishing Alert

### B.1 Create the Email Address for the Threat Actor

In [5]:
email_path = "SCO/Email_Addr/email_addr_THREAT.json"
results_path = "step1/THREAT__email_addr.json"
# 1. invoke the email object using method from ./Utilities/local_make_sco.py
threat_email_addr_list = invoke_make_email_addr_block(email_path, results_path)
# 2. add the record to the in-session bundles and incident lists
bundle_list = bundle_list + threat_email_addr_list
other_object_refs = other_object_refs + threat_email_addr_list
# recover the threat email object in dict form as a global variable for later
threat_email_addr = conv(threat_email_addr_list[0])

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}


### B.2 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

In [6]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
to_email_addr = alert_data["to"]
TR_name = "Trusty Jones"
# 2. Setup variables to hold the dicts
reporter_account = {}
reporter_email_addr = {}
reporter_identity = {}
TR_identity = {}
#
# 3. Retrieve the details for the employees involved
#
for rec in context_identities:
    if rec["type"] == "user-account":
        if rec["account_login"] == reporter_name:
            reporter_account = rec
    if rec["type"] == "email-addr":
        if rec["value"] == to_email_addr:
            reporter_email_addr = rec

for rec in context_identities:
    if rec["type"] == "identity" and rec["identity_class"] == "individual":
        if rec["name"] == TR_name:
            TR_identity = rec
        for email_address in rec["extensions"]["extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498"]["email_addresses"]:
            if email_address["email_address_ref"] == reporter_email_addr["id"]:
                reporter_identity = rec


# add the record to the in-session bundles and lists
bundle_list = bundle_list + [reporter_account, reporter_email_addr, TR_identity]
other_object_refs = other_object_refs + [reporter_account, reporter_email_addr, TR_identity]

### B.3 Make the URL

In [7]:
url_path = "SCO/URL/suspicious_url.json"
results_path = "step1/THREAT__url.json"
hyperlink = alert_data["url"]
# make the email object
url_list = invoke_make_url_block(url_path, results_path, hyperlink)
# add the record to the in-session bundles and lists
bundle_list = bundle_list + url_list
other_object_refs = other_object_refs + url_list
# recover the threat email object in dict form
sus_url = conv(url_list[0])

{
    "type": "url",
    "spec_version": "2.1",
    "id": "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
    "value": "https://www.northkorea.nk/we/are/mad/"
}


### B.4 Make the Email Message

In [8]:
msg_path = "SCO/Email_Message/suspicious_email_msg.json"
results_path = "step1/SUSS__email_msg.json"
from_ref = threat_email_addr
to_ref = [reporter_email_addr]
cc_ref = []
bcc_ref = []
# make the email object
e_msg_list = invoke_make_e_msg_block(msg_path, results_path, from_ref, to_ref, cc_ref, bcc_ref)
# add the record to the in-session bundles and lists
bundle_list = bundle_list + e_msg_list
other_object_refs = other_object_refs + e_msg_list
# recover the threat email object in dict form
sus_msg = conv(e_msg_list[0])

{
    "type": "email-message",
    "spec_version": "2.1",
    "id": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "is_multipart": false,
    "date": "2020-10-19T01:01:01Z",
    "from_ref": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "to_refs": [
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b"
    ],
    "subject": "we are coming for you",
    "body": "some bad stuff written here"
}


### B.4 Make the "derived-from" SRO

In [9]:
# make the email object
relationship_type = "derived-from"
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_rsults_path = "step1/SRO_derived.json"
derived_from_SRO_list = invoke_sro_block(sro_data_path, sro_rsults_path, sus_msg, sus_url, relationship_type)
# add the record to the in-session bundles and lists
bundle_list = bundle_list + derived_from_SRO_list
other_object_refs = other_object_refs + derived_from_SRO_list
# recover the derived URL to Email Message relationship object in dict form
sus_relation = conv(derived_from_SRO_list[0])

{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--e995743f-bf73-49a1-b844-665fb9cf38ab",
    "created": "2023-12-20T13:15:06.038289Z",
    "modified": "2023-12-20T13:15:06.038289Z",
    "relationship_type": "derived-from",
    "source_ref": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "target_ref": "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7"
}


## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [10]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(threat_email_addr["id"])
observed_ids.append(reporter_account["id"])
observed_ids.append(reporter_email_addr["id"])
observed_ids.append(sus_url["id"])
observed_ids.append(sus_msg["id"])
observed_ids.append(sus_relation["id"])
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step1/observation-alert.json"
# 3. Invoke the Make Observed Data Block
obs_1 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
bundle_list = bundle_list + obs_1
other_object_refs = other_object_refs + obs_1

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--0dddd6b7-9bc8-42bf-8b21-cbec6bf1bc0b",
    "created": "2023-12-20T13:15:06.075401Z",
    "modified": "2023-12-20T13:15:06.075401Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
        "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
        "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
        "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
        "relationship--e995743f-bf73-49a1-b844-665fb9cf38ab"
    ]
}


## D. Create the Alert Sighting

Tbis then represents the sighting of potential phishing email as a group of elements

### D.1 Create a Indicator with a Pattern for the Phishing Email
but set its status to `unkown` to start with

In [11]:
# 1. Setup pattern for indicator
pat1 = "[email-addr:value = '" + threat_email_addr["value"] + "' AND email:subject = '" + sus_msg["subject"] + "']"
# 2. Setup path to form and results
ind_path ="SDO/Indicator/indicator_alert.json"
results_path ="step1/indicator_alert.json"
# 3. Invoke the Make Observed Data Block
ind_1 = invoke_make_indicator_block(ind_path, results_path, pattern=pat1)
# 4. Add the record to the in-session bundles and lists
bundle_list = bundle_list + ind_1
other_object_refs = other_object_refs + ind_1

{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--e9e0151c-2bf7-4cbd-ab31-889dabf3130b",
    "created": "2023-12-20T13:15:06.098785Z",
    "modified": "2023-12-20T13:15:06.098785Z",
    "name": "Potential Phishing Email",
    "description": "Suspicious email reported by user",
    "indicator_types": [
        "unknown"
    ],
    "pattern": "[email-addr:value = 'evil@northkorea.nk' AND email:subject = 'we are coming for you']",
    "pattern_type": "stix",
    "pattern_version": "2.1",
    "valid_from": "2023-12-20T13:15:06.098785Z"
}


### D.2 Create the Sighting
Connecting the:
- observed-data object
- indicator object
- the reporter identity as the location

In [12]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_alert.json"
results_path ="step1/sighting_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighted = ind_1[0]["id"]
observed_list = [obs_1[0]["id"]]
where_list = [reporter_identity["id"]]
# 2. Invoke the Make Observed Data Block
sight1 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=where_list)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + sight1
other_object_refs = other_object_refs + sight1

{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--96eb2eb9-d161-4673-b9ca-0faf08340c35",
    "created": "2023-12-20T13:15:06.13802Z",
    "modified": "2023-12-20T13:15:06.13802Z",
    "count": 1,
    "sighting_of_ref": "indicator--e9e0151c-2bf7-4cbd-ab31-889dabf3130b",
    "observed_data_refs": [
        "observed-data--0dddd6b7-9bc8-42bf-8b21-cbec6bf1bc0b"
    ],
    "where_sighted_refs": [
        "identity--cf7cd317-8322-4e44-bd86-0386dfcf50ba"
    ]
}


## E. Create the Event Objects

### E.1 First Create the Event
Tbis Event represents the sighting of potential phishing email as a group of elements,no real Impact yet

In [13]:
# 1. Setup path to form and results
event_data_path ="SDO/Event/event_alert.json"
results_path ="step1/event_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighting = sight1[0]["id"]
# 2. Invoke the Make Observed Data Block
event_1 = invoke_make_event_block(event_data_path, results_path, changed_objects=None,sighting_refs=sighting)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + event_1
event_refs = event_refs + event_1

{
    "type": "event",
    "spec_version": "2.1",
    "id": "event--869ea5d9-c602-4e9d-b2d4-4119cc8bc7c7",
    "created": "2023-12-20T13:15:06.165026Z",
    "modified": "2023-12-20T13:15:06.165026Z",
    "description": "Suspicious email reported by user",
    "end_time": "2023-12-20T13:15:06.165026Z",
    "event_types": [
        "dissemination-phishing-emails"
    ],
    "name": "Potential Phishing Email",
    "sighting_refs": [
        "sighting--96eb2eb9-d161-4673-b9ca-0faf08340c35"
    ],
    "start_time": "2023-12-20T13:15:06.165026Z"
}



### E.2 Then Create the Sequence Object for the Event

The Sequence object is created to place the event in a sequence, in this case a single-step

In [14]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_event_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "event"
sequenced_object = event_1[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_E_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_E_1
sequence_refs = sequence_refs + seq_E_1

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--d815cbdf-37c1-44bc-989b-87dea4b2b252",
    "created": "2023-12-20T13:15:06.197612Z",
    "modified": "2023-12-20T13:15:06.197612Z",
    "sequenced_object": "event--869ea5d9-c602-4e9d-b2d4-4119cc8bc7c7",
    "sequence_type": "event",
    "step_type": "single_step"
}



### E.3 Since it is the First Sequence, Create the Start Sequence Object and Connect it

The Sequence object is created to place the event in a sequence, in this case a start-step

In [15]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_event_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "event"
on_completion = seq_E_1[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_E_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_E_0
sequence_start_refs = sequence_start_refs + seq_E_0

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--a4e3fd06-0ba6-49c9-aec4-e96ab813cb25",
    "created": "2023-12-20T13:15:06.229304Z",
    "modified": "2023-12-20T13:15:06.229304Z",
    "sequence_type": "event",
    "step_type": "start_step",
    "on_completion": "sequence--d815cbdf-37c1-44bc-989b-87dea4b2b252"
}


## F. Create the Task Objects

Tbis Task represents the first Task to clarify the situation first,no real Impactyet

In [16]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_alert.json"
results_path ="step1/task_alert.json"
# 2. Invoke the Make Observed Data Block
task_1 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + task_1
task_refs = task_refs + task_1

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--8e895b6d-891a-410b-8474-c689d5168d7e",
    "created": "2023-12-20T13:15:06.25734Z",
    "modified": "2023-12-20T13:15:06.25734Z",
    "description": "Suspicious email reported by user",
    "end_time": "2023-12-20T13:15:06.25734Z",
    "name": "Potential Phishing Email",
    "start_time": "2023-12-20T13:15:06.25734Z"
}


### F.2 Then Create the Sequence Object for the Task

The Sequence object is created to place the Task in a sequence, in this case a single-step

In [17]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_1[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_T_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_T_1
sequence_refs = sequence_refs + seq_E_1

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--037db86a-5cc3-4b85-a17a-9500a3c54ad9",
    "created": "2023-12-20T13:15:06.291913Z",
    "modified": "2023-12-20T13:15:06.291913Z",
    "sequenced_object": "task--8e895b6d-891a-410b-8474-c689d5168d7e",
    "sequence_type": "task",
    "step_type": "single_step"
}



### F.3 Then Create the Sequence Start Object for the Tasks

The Sequence object is created to place the Task in a sequence, in this case a start-step

In [18]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_task_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "task"
on_completion = seq_T_1[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_T_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_T_0
sequence_start_refs = sequence_start_refs + seq_T_0

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--2d5fdea0-9404-4459-b6d0-174ec56b9f7b",
    "created": "2023-12-20T13:15:06.327113Z",
    "modified": "2023-12-20T13:15:06.327113Z",
    "sequence_type": "task",
    "step_type": "start_step",
    "on_completion": "sequence--037db86a-5cc3-4b85-a17a-9500a3c54ad9"
}


## G. Create the Incident Object

Create the Incident object and assign all of the objets to it

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident